In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import json
import seaborn as sns
import re
from IPython.core.display import HTML

In [2]:
traffic = pd.read_csv('../data/traffic_data.csv')
traffic_state = pd.read_csv('../data/traffic_fatalities_state.csv')

In [3]:
traffic['location'] = traffic['zipcode'].apply(lambda x: 'Gulfport Beach' if x == 39501 else ('Isle of Palms' if x == 29451 else 'Panama City Beach' if x == 32407 else 'Dauphin Island' if x == 36528 else 'Orange Beach' if x == 36561 else 'Miramar Beach' if x == 32550 else 'Tybee Island'))

In [4]:
traffic['crach_type'] = traffic['crach_type'].apply(lambda x: 'Alcohol Impaired' if x == '(1) Alcohol-Impaired Driving (BAC=.08+) Fatalities'\
                                                   else ('Single Vehicle' if x == '(2) Single Vehicle Crash Fatalities'\
                                                        else 'Large Truck Involved' if x == '(3) Large Truck Involved Crash Fatalities'\
                                                        else 'Speeding' if x == '(4) Speeding Involved Crash Fatalities'\
                                                        else 'Rollover' if x == '(5) Rollover Involved Crash Fatalities '\
                                                        else 'Roadway Departure' if x == '(6) Roadway Departure Involved Crash Fatalities'\
                                                        else 'Intersection' if x == '(7) Intersection (or Intersection Related) Crash Fatalities'\
                                                        else 'Bicyclist' if x == 'Bicyclist (or Other Cyclist) Fatalities'\
                                                        else 'Light Truck' if x == 'Light Truck Occupant Fatalities'\
                                                        else 'Motorcyclist' if x == 'Motorcyclist Fatalities'\
                                                        else 'Passenger Car' if x == 'Passenger Car Occupant Fatalities'\
                                                        else 'Pedestrian' if x == 'Pedestrian Fatalities'\
                                                        else 'Total Fatalities'))

In [5]:
traffic = traffic.rename(columns = {'crach_type':'crash_type'})

In [6]:
traffic

,zipcode,crash_type,2017,2018,2019,2020,2021,"2017 (per 100,00)","2018 (per 100,00)","2019 (per 100,00)","2020 (per 100,00)","2021 (per 100,00)",location
0,36528,Total Fatalities,75,74,84,61,79,18.11,17.86,20.31,14.73,19.12,Dauphin Island
1,36528,Alcohol Impaired,19,21,28,16,29,4.59,5.07,6.77,3.86,7.02,Dauphin Island
2,36528,Single Vehicle,41,37,44,33,37,9.90,8.93,10.64,7.97,8.96,Dauphin Island
3,36528,Large Truck Involved,7,8,13,7,11,1.69,1.93,3.14,1.69,2.66,Dauphin Island
4,36528,Speeding,14,17,18,13,24,3.38,4.10,4.35,3.14,5.81,Dauphin Island
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,29451,Passenger Car,29,20,23,17,18,7.20,4.91,5.57,4.15,4.36,Isle of Palms
87,29451,Light Truck,18,12,19,15,17,4.47,2.95,4.60,3.66,4.12,Isle of Palms
88,29451,Motorcyclist,9,9,8,11,14,2.23,2.21,1.94,2.69,3.39,Isle of Palms
89,29451,Pedestrian,14,23,18,25,17,3.48,5.65,4.36,6.11,4.12,Isle of Palms


In [7]:
traffic_state['2010'] = traffic_state['2010'].str.replace(',', '').astype(int)
traffic_state['2011'] = traffic_state['2011'].str.replace(',', '').astype(int)
traffic_state['2012'] = traffic_state['2012'].str.replace(',', '').astype(int)
traffic_state['2013'] = traffic_state['2013'].str.replace(',', '').astype(int)
traffic_state['2014'] = traffic_state['2014'].str.replace(',', '').astype(int)
traffic_state['2015'] = traffic_state['2015'].str.replace(',', '').astype(int)
traffic_state['2016'] = traffic_state['2016'].str.replace(',', '').astype(int)
traffic_state['2017'] = traffic_state['2017'].str.replace(',', '').astype(int)
traffic_state['2018'] = traffic_state['2018'].str.replace(',', '').astype(int)
traffic_state['2019'] = traffic_state['2019'].str.replace(',', '').astype(int)
traffic_state['2020'] = traffic_state['2020'].str.replace(',', '').astype(int)

In [8]:
traffic_state

,State,type,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Alabama,Highway Fatalities,862,895,865,853,820,850,1083,948,953,930,934,NaN,NaN
1,Alabama,Rail fatalities,17,17,10,18,21,15,12,24,15,15,13,21.0,20.0
2,Alabama,Recreational boating fatalities,20,19,17,10,13,21,14,21,17,28,19,10.0,NaN
3,Alabama,Transit Fatalities,0,0,0,0,1,0,0,0,0,0,0,0.0,NaN
4,Florida,Highway Fatalities,2444,2400,2431,2403,2494,2938,3176,3116,3135,3185,3331,NaN,NaN
5,Florida,Rail fatalities,44,32,30,35,34,44,44,64,47,55,39,60.0,71.0
6,Florida,Recreational boating fatalities,69,61,50,58,70,52,70,66,57,62,72,61.0,NaN
7,Florida,Transit Fatalities,14,12,13,15,8,8,6,8,7,10,7,17.0,NaN
8,Georgia,Highway Fatalities,1247,1226,1192,1180,1164,1432,1556,1540,1505,1492,1664,NaN,NaN
9,Georgia,Rail fatalities,29,19,21,28,25,24,23,19,29,23,35,21.0,19.0
